# Générateur de partition à la Bach

## Chaînes de Markov
Ici on travaille avec des fichiers midi de trois partitions de Bach :
> Prelude et fugue en Ut majeur (BWV 846)

> Prelude et fugue en Ut majeur (BWV 870)

> Prelude et fugue en Ut mineur (BWV 847)

### 1. Lecture des fichiers midi
Le but est d'obtenir un array de note (string) facilement manipulable.

In [2]:
#Librarie pour comprendre midi
from music21 import *

In [55]:
#Fonction pour lire les fichiers midi
def read_midi(file):
    
    print("Chargement du fichier:",file)
    
    notes=[]
    notes_to_parse = None
    
    #Analyse du fichier
    midi = converter.parse(file)
  
    #Regroupement si plusieurs instruments (peu utile mais on sait jamais)
    s2 = instrument.partitionByInstrument(midi)

    #Boucle sur chaque instrument
    for part in s2.parts:
    
        #On ne récupère que le piano
        if 'Piano' in str(part): 
        
            notes_to_parse = part.recurse() 
      
            #On détermine si l'élement analysé est un accord ou une note solitaire
            for element in notes_to_parse:
                
                #note
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                
                #accord
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(notes)

In [57]:
#Pour récupérer les fichiers
import os

#Numpy -classique-
import numpy as np

#On précise le chemin où se trouvent les fichiers .mid
path='_res/'

#On lit le nom de fichiers
files=[i for i in os.listdir(path) if i.endswith(".mid")]

#On lit chaque fichier
notes_array = np.array([read_midi(path+i) for i in files])

Chargement du fichier: _res/bach_prelude.mid
Chargement du fichier: _res/bach_prelude2.mid
Chargement du fichier: _res/back_prelude3_minor.mid


In [59]:
#Passage d'un array 2D à un array 1D : on met les partitions les unes à la suite des autres
notes_ = [element for note_ in notes_array for element in note_]

### 2. Mise en place de la chaîne de Markov
A partir de l'array *notes_*, on va créer un dictionnaire qui se présente sous la forme suivante :<br>

| Clé | Valeur |
| :- | :- |
| Note_1  | [Liste des notes qui suivent directement Mot_1 dans le texte original] |
| Note_2  | [Liste des notes qui suivent directement Mot_2 dans le texte original] 

In [22]:
from collections import defaultdict

def markov_chain(partition):
    '''L'entrée est une string et la sortie est un dictionnaire avec chaque note de l'array en clé et 
    la liste des notes qui suivent cette clé dans l'array en valeur. '''
    
    # On initialise un dictionnaire 'defaultdict'
    m_dict = defaultdict(list)
    
    # Pour chaque couple de notes qui se suivent on met la première en clé s'elle n'existe pas déjà, et on ajoute la deuxième à la liste de notes en valeur.
    for current_note, next_note in zip(partition[0:-1], partition[1:]):
        m_dict[current_note].append(next_note) #possible grâce au defaultdict, la clé est créée si elle n'existe pas encore

    # On convertit en dictionnaire python
    m_dict = dict(m_dict)
    return m_dict

In [51]:
# Création du dictionnaire pour Bach
bach_dict = markov_chain(notes_)

### 3. Génération de la partition
Création d'une fonction qui va générer la partition. Elle prends deux arguments :
* Le dictionnaire tout juste créé
* Le nombre de notes que doit contenir la partition

On obtient un array de notes que l'on convertit en midi et que l'on sauvegarde dans un dossier.

In [31]:
import random

def generate_sheet(chain, count=120):
    '''chain : dictionnaire généré
       count : nombre de mots dans la phrase à générer'''
    word1 = random.choice(list(chain.keys()))
    partition = [word1]

    # Génère le second mot d'après la liste de valeur du premier mot. Le nouveau mot devient le "premier mot". On répète.
    for i in range(count-1):
        word2 = random.choice(chain[word1])
        word1 = word2
        partition.append(word2)

    return(partition)

In [60]:
def convert_to_midi(prediction_output):
   
    offset = 0
    output_notes = []

    # crée les objets notes et accords à partir de ce que le modèle a génèré
    for pattern in prediction_output:
        
        # si pattern est un accord \ex : '1.6.9'\
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # si pattern est une note \ex : 'D4'\
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # on change l'offset à chaque iteration pour que les notes se succèdent
        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')

In [53]:
convert_to_midi(generate_sheet(bach_dict))

### 4. Résultats
On a fait trois générations différentes.<br>

La première utilise le premier prélude seulement en entrée. La partition obtenue est chaotique mais tient la route en terme de cohérence mélodique. Voir *02_Markov_Chains_One_Input.mid*<br>
La deuxième prend deux préludes qui sont dans la même tonalité : Ut Majeur. Le nombre d'incohérence augmente mais le résultat reste satisfaisant. Voir *02_Markov_Chains_Two_Input.mid*
La deuxième prend trois préludes qui sont dans des tonalités différentes : Ut Majeur et Ut Mineur. Cette fois-ci, le modèle arrive à sa limite avec des incohérences très fréquentes, liées aux différences de mélodies entre les morceaux. Voir *02_Markov_Chains_Three_Input.mid*

<br><br>
Le modèle de Markov tel que présenté a le problème majeur de ne travailler que sur la logique de la mélodie et pas sur celle de l'harmonie. De plus, le choix de la note suivante ne prend en compte que la précédente.

In [72]:
import IPython.display as ipd
ipd.Audio('_output/02_Markov_Chains_One_Input.mp3')

In [70]:
ipd.Audio('_output/02_Markov_Chains_Two_Input.mp3')

In [71]:
ipd.Audio('_output/02_Markov_Chains_Three_Input.mp3')